# Building a custom Widget - Hello World
The widget framework is built on top of the Comm framework (short for communication). The Comm framework is a framework that allows the kernel to send/receive JSON messages to/from the front-end. 

* Interact
* Widgets
* COMM
* Websockets/Zeromq

To create a custom widget, you need to define the widget both in the browser and the python kernel.

## Building a Custom Widget
To get started, you'll create a simple hello world widet. Later you'll build on this foundation to make more complex widgets. 

## Python Kernel
### DOMWidget and Widget

To define a widget, you must inherit from the Widget or DOMWidget base class. If you intend for your widget to be displayed int he Jupyter notebook, you'll want to inherit from the DOMWidget. The DOMWidget class itself inherits fro the Widget class. The Widget class is useful for cases in which the Widget is not meant to be displayed directly in the notebook, but instead as a child of another rendering environment. For example, if you wanted to create a three.js widget (a poplar WebGL library), you would implement the rendering window as a DOMWidget and any 3D objects or lights meant to be rendered in that window as Widgets. 

### _View_name
Inheriting from the DOMWidget does not tell the widget framework waht front end widget to associate with your back end widget 

Instead, you must tell it yourself by defining secially named trait attributes, ``view_name`` and ``_view_module`` (as seen below) and optionaly ``_model``. 


In [27]:
import ipywidgets as widgets
from traitlets import Unicode, validate

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)

### sync=True traitlets
Traitlets is an IPython library for defining type-safe properties on configurable objects. For this tutorial you do not need to worry about the *configurable* piece of the traitlets machinery. The ``sync=True`` keyword argument tells the widget framework to handle synchronizing that value to the browser. Without ``sync=Tue``, the browser would have no knowledge of ``_view_name`` or ``_view_module``. 

### Other traitlet types
Unicode, used for ``view_name``, is not the only Traitlet type, there are many more, some of which are listed below:
* Any
* Bool
...

## Front End (JavaScript)
### Models and views
The IPython widget framework front end relies heavily on Backbone.js. Backbone.js is a MVC (model view controller) framework. Widgets defined int he back end are automatically synchronized with generic Backbone.js models on the front end. The traitlets are added to the  front end instance automatially on the first state push. The ``_view_name`` trait that you defined earlier is used by the widget framework to create the corresponding Backvone.js view and link that view to the model.

### Import jupyter-js-widgets
You first need to import the ``jupyter-js-widgets`` module. To import modules, use the ``define`` method of require.js (as seen below).

In [9]:
%%javascript
define('hello', ['jupyter-js-widgets'], function(widgets){
    
});

<IPython.core.display.Javascript object>

### Define the view 
Next define your widget view class. Inherit from the ``DOMWidgetView`` by using the ``.extend`` method. 

In [10]:
%%javascript
require.undef('hello');

define('hello', ['jupyter-js-widgets'], function(widgets){
    // define the HelloView
    var HelloView = widgets.DOMWidgetView.extend({
        
    });
    
    return {
        HelloView: HelloView
    }
});

<IPython.core.display.Javascript object>

### Render method
Lastly, override the base ``render`` method of the view to define custom rendering logic. A handle to the widget's default DOM element can be acquired via ``this.el``. The ``el`` property is the DOM element associated with the view. 

In [11]:
%%javascript
require.undef('hello');

define('hello', ['jupyter-js-widgets'], function(widgets){
    // define the HelloView
    var HelloView = widgets.DOMWidgetView.extend({
        // Render the view
        render: function(){
            this.el.textContent = 'Hello World!';
        },
    });
    
    return {
        HelloView: HelloView
    }
})

<IPython.core.display.Javascript object>

## Test
You should be able to display your widget just like any other widget now. 


In [12]:
HelloWidget()

## Making the widget stateful
There is not much that you can do with the above example that you can't do with the IPyhon display framework. To change this, you will make the widget stateful. Instead of displaying a static "hello world" message, it will display a string set by the back end. First, you will need to adda traitlet in the back end. Use the name of ``value`` to stay consistent with the rest of the widget framework and to allow your widget to be used with interact. 

In [31]:
class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    value = Unicode('Hello World!').tag(sync=True)

        
    def getCount(self):
        return('current count')

### Accessing the model from the view
To access the model associated with a view instance, use the `model` property of the view. `get` and `set` methods are used to interact with the Backbone model. `get` is trivial, however you have to be careful when using `set`. After calling the model `set` you need to call the view's `touch` method. This associates the `set` operation with a particular view so output will be routed to the correct cell. The model also has an `on` method, which allows you to listen to events triggered by the model (like value changes).

### Rendering model contents
By replacing the string literal with a call to `model.get`, the view will now display the value of the back end upon display. Howeer, it will not update itself to a new value when the value changes. 

In [32]:
%%javascript
require.undef('hello');

define('hello', ["jupyter-js-widgets"], function(widgets) {

    var HelloView = widgets.DOMWidgetView.extend({

        render: function() {
            this.el.textContent = this.model.get('value');
        },
    });

    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [16]:
HelloWidget(value='some custom string')

### Dynamic updates
To get the view to update itself dynamically, register a function to update the view's value when the model's `value` property chages. This can be done using the `model.on` method. The `on` method takes three parameters, an event name, callback handle, and callback context. The Backbone event named `change` will fire whenever the model changes. By appending `:value` to it, you tell Backbone to only listen to the change event of the `value` property (as seen below). 


In [17]:
%%javascript
require.undef('hello');

define('hello', ['jupyter-js-widgets'], function(widgets){
    var HelloView = widgets.DOMWidgetView.extend({
        render: function(){
            this.value_changed();
            // event name, callback handle, callback context
            this.model.on('change:value', this.value_changed, this);
        },
        
        value_changed: function(){
            this.el.textContent = this.model.get('value');
        },
    });
    
    return {
        HelloView : HelloView
    };
});

<IPython.core.display.Javascript object>

In [33]:
w = HelloWidget()

In [34]:
w


In [35]:
w.value = 'something'

### Conclusion
The example above dumps the value directly to the DOM. There is no way for you to interact with this dumped data in teh front-end. To create an example that accepts input, you will have to do something more than blindly dumping the contents of value into the DOM. 

In the next section of the tutorial, you will build a date picker to display and accept input in the front-end.

In [36]:
w.getCount()

'current count'